# Titanic - Machine Learning from Disaster

- Vamos utilizar os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - É um dataset de **competição**
    - O resultado é avaliado através da **acurácia**:
        - _"Sua pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido como acurácia."_

In [1]:
import pandas as pd

In [3]:
treino = pd.read_csv('train.csv')

In [4]:
teste = pd.read_csv('test.csv')

> Como já houve uma tentativa para esse desafio, vamos apenas replicar algumas etapas e tentar melhorar a precisão de nossa previsão.

In [5]:
# Eliminando as colunas com elevada cardinalidade
treino = treino.drop(['Name','Ticket','Cabin'],axis=1)
teste = teste.drop(['Name','Ticket','Cabin'],axis=1)

In [7]:
# Usando a mediana para imputar os valores nulos na coluna de idade
treino.loc[treino.Age.isnull(),'Age'] = treino.Age.median()
teste.loc[teste.Age.isnull(),'Age'] = teste.Age.median()

In [8]:
# Tratando a coluna Embarked da base de treino usando a moda 
treino.loc[treino.Embarked.isnull(),'Embarked'] = treino.Embarked.mode()[0]

In [9]:
# E também a coluna Fare da base de teste usando a média
teste.loc[teste.Fare.isnull(),'Fare'] = teste.Fare.mean()

In [10]:
treino.dtypes=='object'

PassengerId    False
Survived       False
Pclass         False
Sex             True
Age            False
SibSp          False
Parch          False
Fare           False
Embarked        True
dtype: bool

In [11]:
treino.columns[treino.dtypes=='object']

Index(['Sex', 'Embarked'], dtype='object')

In [12]:
treino.Sex.value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [13]:
treino.Embarked.value_counts()

Embarked
S    646
C    168
Q     77
Name: count, dtype: int64

### Variável Sex

Vamos transformar a coluna Sex código númerico para que a coluna possa ser usada no modelo de aprendizagem

In [14]:
sex = {'male': 1,
       'female': 0}

In [15]:
treino['MaleCheck'] = treino.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [16]:
treino[['Sex', 'MaleCheck']].value_counts()

Sex     MaleCheck
male    1            577
female  0            314
Name: count, dtype: int64

In [17]:
teste['MaleCheck'] = teste.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [18]:
teste[['Sex', 'MaleCheck']].value_counts()

Sex     MaleCheck
male    1            266
female  0            152
Name: count, dtype: int64

### OneHotEncoder na coluna Embarked: Por quê

- OneHotEncoder é uma técnica de pré-processamento usada em aprendizado de máquina para lidar com variáveis categóricas. Ele converte variáveis categóricas em uma representação binária, criando uma nova coluna para cada categoria com valores binários indicando a presença ou ausência dessa categoria.

- Essa técnica é especialmente interessante por que há um pequeno número de categorias (3) na coluna 'Embarked', melhorando a eficiencia de aprendizado de máquina, Permitindo que o modelo compreenda e utilize efetivamente informações de variáveis categóricas em seus cálculos.

    - Ao criar colunas binárias para cada local de embarque, o modelo pode interpretar de maneira mais eficaz as informações categóricas, melhorando a capacidade preditiva do algoritmo. Essa abordagem é especialmente relevante quando lidamos com um conjunto de dados contendo um número restrito de categorias, contribuindo para a interpretação e desempenho aprimorados do modelo.

In [19]:
from sklearn.preprocessing import OneHotEncoder

In [20]:
# Criando o encoder
ohe = OneHotEncoder(handle_unknown='ignore',dtype='int32')

In [21]:
ohe = ohe.fit(treino[['Embarked']])

In [22]:
# Fazendo a transformação
ohe.transform(treino[['Embarked']]).toarray()

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]])

In [23]:
# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(treino[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())
ohe_df.head(3)

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1


In [24]:
# Concatenando o dataframe 
treino = pd.concat([treino,ohe_df],axis=1)

In [25]:
# Verificando os valores 
treino[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             646
C         1           0           0             168
Q         0           1           0              77
Name: count, dtype: int64

- Agora fazendo o mesmo para a base de teste usando o encoder ohe que criamos acima

In [26]:
ohe_df = pd.DataFrame(ohe.transform(teste[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())

In [27]:
teste = pd.concat([teste,ohe_df],axis=1)

In [29]:
# Verificando os valores
teste[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             270
C         1           0           0             102
Q         0           1           0              46
Name: count, dtype: int64

In [30]:
treino.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,male,22.0,1,0,7.2500,S,1,0,0,1
1,2,1,1,female,38.0,1,0,71.2833,C,0,1,0,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,0,0,1
3,4,1,1,female,35.0,1,0,53.1000,S,0,0,0,1
4,5,0,3,male,35.0,0,0,8.0500,S,1,0,0,1


In [31]:
# Eliminando as colunas que foram tratadas/modificadas
treino = treino.drop(['Sex','Embarked'],axis=1)
teste = teste.drop(['Sex','Embarked'],axis=1)

### Alguns modelos de teste que poderiam ser usados para classificar os presentes dados
- Para começar, vamos testar entre:
    - **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html#classification
    - **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

- train_test_split é uma função da biblioteca Scikit-Learn que tem o objetivo de dividir um conjunto de dados em duas partes: uma para treinamento do modelo e outra para avaliação do desempenho do modelo. A função garante que o modelo generalize bem para novos dados não vistos, prevenindo overfitting. O conjunto de treinamento é usado para treinar o modelo, enquanto o conjunto de teste é reservado para avaliação de desempenho.

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X = treino.drop(['PassengerId','Survived'],axis=1)
# A tabela abaixo atribui o resultado que esperamos, para verificar se há um modelo que possa prever os sobreviventes.
y = treino.Survived

In [34]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

### Criando arvore de classificação

In [35]:
from sklearn import tree

In [46]:
# Criando o classificador
# clf -> classificação
# ac -> arvore de classificação
clf_ac = tree.DecisionTreeClassifier()

In [47]:
# Fazendo o fit com os dados (treinando o modelo)
clf_ac = clf_ac.fit(X_train,y_train)

In [48]:
y_pred_ac = clf_ac.predict(X_val)

### Para o **KNeighborsClassifier**

In [40]:
from sklearn.neighbors import KNeighborsClassifier

In [41]:
# Criando o classificador
# clf -> classificação
# knn -> modelo usado (kneighborsclassifier)
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [42]:
# Fazendo o fit com os dados (treinando o modelo)
clf_knn = clf_knn.fit(X_train,y_train)

In [43]:
y_pred_knn = clf_knn.predict(X_val)

### Para a **Regressão Logística**

In [45]:
from sklearn.linear_model import LogisticRegression

In [50]:
# Criando o classificador
# clf -> classificador
# rl -> Regressão Logística
clf_rl = LogisticRegression(max_iter=1000)

In [51]:
# Fazendo o fit com os dados (treinando o modelo)
clf_rl = clf_rl.fit(X_train,y_train)

In [52]:
y_pred_rl = clf_rl.predict(X_val)

### Para finalizar, vamos avaliar esses modelos treinados:
- Para fazer essa análise, será usado:
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

In [53]:
from sklearn.metrics import accuracy_score

In [54]:
# Verificando acurácia para a árvore de classificação
accuracy_score(y_val, y_pred_ac)

0.752542372881356

In [55]:
# Verificando acurácia para o knn
accuracy_score(y_val, y_pred_knn)

0.7084745762711865

In [56]:
# Para a regressão logística
accuracy_score(y_val, y_pred_rl)

0.8033898305084746

- Desse modo, o modelo de regressão logística se mostrou o mais efetivo para este dataset

### Avaliando a **matriz de confusão**

- Utilizaremos a matriz de confusão afim de validar mais uma vez a ferramenta mais eficiente aplicada ao nosso teste. 
    - O módulo confusion_matrix do Scikit-Learn é utilizado para avaliar o desempenho de modelos de classificação. Ele cria uma tabela que mostra a contagem de acertos e erros do modelo em diferentes categorias

In [57]:
from sklearn.metrics import confusion_matrix

In [58]:
# Para a árvore
confusion_matrix(y_val, y_pred_ac)

array([[149,  39],
       [ 34,  73]], dtype=int64)

In [59]:
# Para o knn
confusion_matrix(y_val, y_pred_knn)

array([[153,  35],
       [ 51,  56]], dtype=int64)

In [60]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

array([[159,  29],
       [ 29,  78]], dtype=int64)

- Após mais uma verificação, o modelo de regressão logística se mostrou o mais efetivo novamente para este dataset

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [61]:
 X_train.head()

,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
354,3,28.0,0,0,7.225,1,1,0,0
414,3,44.0,0,0,7.925,1,0,0,1
210,3,24.0,0,0,7.050,1,0,0,1
446,2,13.0,0,1,19.500,0,0,0,1
565,3,24.0,2,0,24.150,1,0,0,1


In [62]:
teste.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,1,0,1,0
1,893,3,47.0,1,0,7.0000,0,0,0,1
2,894,2,62.0,0,0,9.6875,1,0,1,0
3,895,3,27.0,0,0,8.6625,1,0,0,1
4,896,3,22.0,1,1,12.2875,0,0,0,1


In [63]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_teste = teste.drop('PassengerId',axis=1)

In [64]:
# Utilizando a regressão logística na base de teste
y_pred = clf_rl.predict(X_teste)

In [65]:
# Criando uma nova coluna com a previsão na base de teste
teste['Survived'] = y_pred

In [66]:
teste['Survived']

0      0
1      0
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    0
Name: Survived, Length: 418, dtype: int64

In [67]:
teste[['PassengerId','Survived']]

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [68]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio
base_envio = teste[['PassengerId','Survived']]

In [69]:
# Exportando para um csv
base_envio.to_csv('resultados4.csv',index=False)